In [1]:
import datetime
import time

In [11]:
class Customer:
  def __init__(self, focal_name, size, is_reservation = False, reservation_time = None):
    self.focal_name = focal_name
    self.size = size
    self.is_reservation = is_reservation
    self.reservation_time = reservation_time
    self.arrival_time = datetime.datetime.now()

    self.customer_type = "reservation" if self.is_reservation else "walk-ins"

  def __str__(self):
    return f'Customer: "{self.focal_name}" / number of people: {self.size} / Type: {self.customer_type}: Reservation Time: {self.reservation_time}'

  def setting_priority(self):  #M
    if self.is_reservation:
      priority_status = 0
    else:
      priority_status = 1
    #print(f"customer's priority: {priority_status}")

    if self.reservation_time:
      relevant_time = self.reservation_time
    else:
      relevant_time = self.arrival_time
    #print(f"customer's time: {relevant_time}")
    return (priority_status, relevant_time)

  def __lt__(self, other):
    return self.setting_priority() < other.setting_priority()  #used comparator


In [3]:
class PriorityQueue:
  def __init__(self):
    self._queue = []

  def enqueue(self, customer_data):
    if not isinstance(customer_data, Customer):
      raise TypeError("Only Customer can be enqueued.")
    self._queue.append(customer_data)
    self._queue.sort()
    print(f"Enqueue: {customer_data}.\n No {len(self._queue)}. Current queue: {[str(i) for i in self._queue]}")

  def dequeue(self):
    if self.is_empty():
      return f"Queue is empty."
    popped_person = self._queue.pop(0)
    print(f"Dequeued: {popped_person.focal_name}.\n Current queue: {[str(i) for i in self._queue]}")
    return popped_person

  def is_front(self):
    #print(f"The front queue is {self._queue[0]}\n")
    return self._queue[0]

  def is_empty(self):
    return len(self._queue) == 0

  def queue_size(self):
    return len(self._queue)

In [12]:
class Table:
  def __init__(self, name, capacity):
    self.name = name
    self.is_occupied = False
    self.capacity = capacity
    self.status = "available"    #available/free/occupied
    self.clean = True
    #self.customer = None
    self.current_customer = None
    self.order_stage = None  #seated/ordered/served/dessert/bill/paid
    self.seated_time = None

  def __str__(self):
    return f'Table name: {self.name}, Capacity: {self.capacity - 1} - {self.capacity}'

  def is_available(self, customer_size):
    return not self.is_occupied and self.clean and customer_size <= self.capacity and self.status == "available" #available for customer

  def seat_customer(self, customer):
    if self.is_available(customer.size):
      self.is_occupied = True
      self.status = "occupied"
      self.clean = False   #Table becomes dirty once occupied
      self.current_customer = customer
      self._seated_time = datetime.datetime.now()
      self.order_stage = "seated"
      print(f'{customer.focal_name}:{customer.size} is seated at Table {self.name} with capacity {self.capacity}.\n')
      return True
    else:
      print(f"Table {self.name} cannot accomodate customer {customer.focal_name} due to insufficient capacity.\n")
      return False

  def free_table(self):
    self.status = "dirty"
    if self.is_occupied:
      self.is_occupied = False
      self.order_stage = "dirty" # Stage indicates it needs cleaning
      self.seated_time = None # Reset seated time
      print(f"Table {self.name} is now free but dirty. Last customer: {self.current_customer}")
      self.current_customer = None
    else:
      print(f"Table is already free.")

  def clean_table(self):
    self.status = "available"
    self.clean = True
    self.order_stage = "cleaned"
    print(f"Table {self.name} is now clean and available.")

  def update_table_status(table, stage):
    order_stages = [
        "seated", "ordered", "food_served",
        "dessert", "bill_requested", "paid",
        "dirty", "cleaned"
    ]

    if stage not in order_stages:
        print(f"{stage} is not valid.")
        return

    if stage == "paid":
        table.status = "dirty"
        table.customer = None
        table.order_stage = "dirty"
        print(f"Table {table.name} has completed payment.\n Status: dirty.")
        return

    if stage == "cleaned":
        table.clean_table()
        return

    table.order_stage = stage
    print(f"Table {table.name} updated to stage: {stage}.")

In [5]:
class Restaurant:
    def __init__(self, tables, cust_queue):
        self.tables = tables
        self.cust_queue = cust_queue
        self.cust_waiting_table = []

    def setup_tables(self):
        for i in range(6):
            self.tables.append(Table(f"August{i+1}", 2))  #6 August tables
        for i in range(4):
            self.tables.append(Table(f"Spring{i+1}", 4))   #4 Spring tables
        for i in range(2):
            self.tables.append(Table(f"Winter{i+1}", 6))   # 2 Winter tables
        self.tables.append(Table("Fall1", 8))  #1 Fall table

    def free_table(self, table_name):
        for table in self.tables:
            if table.name == table_name:
                table.free_table()
                return

    def clean_table(self, table_name):
        for table in self.tables:
            if table.name == table_name:
                table.clean_table()
                return

    def assign_table(self):
        while not self.cust_queue.is_empty():
            peek_cust = self.cust_queue.is_front()
            print(f"Checking: Customer name: {peek_cust.focal_name} / size: {peek_cust.size}")

            suitable_table = None
            if peek_cust.size <= 2:
                i = 0
                while i < 6:
                    if self.tables[i].is_available(peek_cust.size):
                        suitable_table = self.tables[i]
                        break
                    i += 1

            elif peek_cust.size == 3 or peek_cust.size == 4:
                j = 6
                while j < 10:
                    if self.tables[j].is_available(peek_cust.size):
                        suitable_table = self.tables[j]
                        break
                    j += 1

            elif peek_cust.size == 5 or peek_cust.size == 6:
                k = 10
                while k < 12:
                    if self.tables[k].is_available(peek_cust.size):
                        suitable_table = self.tables[k]
                        break
                    k += 1

            elif peek_cust.size == 7 or peek_cust.size == 8:
                l = 12
                while l < 13:
                    if self.tables[l].is_available(peek_cust.size):
                        suitable_table = self.tables[l]
                        break
                    l += 1

            else:
                print('Too many people. May I request you to split the people.')
                break

            if suitable_table:
                seated_customer = self.cust_queue.dequeue()
                suitable_table.seat_customer(seated_customer)

            else:
                print(f"No suitable table available right now for {peek_cust.focal_name}.")
                self.cust_waiting_table.append(self.cust_queue.dequeue())
                print(f"Customer having to wait table: {[str(i) for i in self.cust_waiting_table]}")
                break

    def print_status(self):
        for table in self.tables:
            customer_info = table.current_customer.focal_name if table.current_customer else "None"
            print(f"{table.name}:\n  Status: {table.status}\n  Clean: {table.clean}\n  Order Stage: {table.order_stage}\n  Customer: {customer_info}\n")
        print("-------------------------\n")

In [10]:
if __name__ == "__main__":
    restaurant = Restaurant([], PriorityQueue())

    # Set up tables
    restaurant.setup_tables()
    for table in restaurant.tables:
        print(table)
    print("---------------------------\n")

    # --- Simulation Scenario ---

    print("\n--- Adding initial customers to the queue ---")
    customer1 = Customer("Alice", 2, is_reservation=True, reservation_time=datetime.datetime.now() + datetime.timedelta(minutes=30))
    customer2 = Customer("Bob", 4, is_reservation=True, reservation_time=datetime.datetime.now() + datetime.timedelta(minutes=10))
    customer3 = Customer("Charlie", 3, is_reservation=True, reservation_time=datetime.datetime.now() + datetime.timedelta(minutes=45))
    customer4 = Customer("David", 2)
    customer5 = Customer("Eve", 5)
    customer6 = Customer("Frank", 1)
    customer7 = Customer("Grace", 1)
    customer8 = Customer("Heidi", 1)
    customer9 = Customer("Ivan", 1)
    customer10 = Customer("Judy", 8)

    # Add initial customers to queue
    restaurant.cust_queue.enqueue(customer1)
    restaurant.cust_queue.enqueue(customer2)
    restaurant.cust_queue.enqueue(customer3)
    restaurant.cust_queue.enqueue(customer4)
    restaurant.cust_queue.enqueue(customer5)
    restaurant.cust_queue.enqueue(customer6)
    restaurant.cust_queue.enqueue(customer7)
    restaurant.cust_queue.enqueue(customer8)
    restaurant.cust_queue.enqueue(customer9)
    restaurant.cust_queue.enqueue(customer10)

    print("\n--- First attempt to assign tables ---")
    restaurant.assign_table()

    print("\n--- Restaurant Status ---")
    restaurant.print_status()

    # Simulate some time passing and table progression
    print("\n--- Simulating order and payment for some tables ---")
    # Let's say Alice (August1) is seated and orders
    if restaurant.tables[0].current_customer and restaurant.tables[0].current_customer.focal_name == "Alice":
        restaurant.tables[0].update_table_status("ordered")
    time.sleep(1)

    # Bob (Spring1) is seated and served food
    if restaurant.tables[6].current_customer and restaurant.tables[6].current_customer.focal_name == "Bob":
        restaurant.tables[6].update_table_status("food_served")
    time.sleep(1)

    # Frank (August2) is seated and finishes dessert
    if restaurant.tables[1].current_customer and restaurant.tables[1].current_customer.focal_name == "Frank":
        restaurant.tables[1].update_table_status("dessert")
    time.sleep(1)

    # Frank requests the bill and pays
    if restaurant.tables[1].current_customer and restaurant.tables[1].current_customer.focal_name == "Frank":
        restaurant.tables[1].update_table_status("bill_requested")
        time.sleep(1)
        restaurant.tables[1].update_table_status("paid") # Table becomes dirty

    print("\n--- After some table status updates, trying to assign again ---")
    restaurant.assign_table()
    restaurant.print_status()

    # Clean the dirty table
    print("\n--- Cleaning August2 table ---")
    restaurant.clean_table("August2")

    print("\n--- Attempting to assign tables after cleaning ---")
    restaurant.assign_table()
    restaurant.print_status()

    print("\n--- Simulating more time and another table freeing up ---")
    # Let's say Alice finishes and pays
    if restaurant.tables[0].current_customer and restaurant.tables[0].current_customer.focal_name == "Alice":
        restaurant.tables[0].update_table_status("paid")
        time.sleep(1)
        restaurant.clean_table("August1")

    print("\n--- Adding a new batch of customers ---")
    customer11 = Customer("Zoe", 2)
    customer12 = Customer("Yara", 4)
    customer13 = Customer("Xavier", 6, is_reservation=True, reservation_time=datetime.datetime.now() + datetime.timedelta(minutes=5)) # Reservation soon
    customer14 = Customer("Wendy", 3)
    customer15 = Customer("Victor", 2)
    customer16 = Customer("Uma", 1)

    restaurant.cust_queue.enqueue(customer11)
    restaurant.cust_queue.enqueue(customer12)
    restaurant.cust_queue.enqueue(customer13)
    restaurant.cust_queue.enqueue(customer14)
    restaurant.cust_queue.enqueue(customer15)
    restaurant.cust_queue.enqueue(customer16)

    print("\n--- Attempting to assign tables after new customer arrivals ---")
    restaurant.assign_table()
    restaurant.print_status()

    print("\n--- Simulating more table turnover and final assignments ---")
    # Let's free up a Spring table
    if restaurant.tables[6].current_customer and restaurant.tables[6].current_customer.focal_name == "Bob":
        restaurant.tables[6].update_table_status("paid")
        time.sleep(1)
        restaurant.clean_table("Spring1")

    # Free up the Fall table
    if restaurant.tables[12].current_customer and restaurant.tables[12].current_customer.focal_name == "Judy":
        restaurant.tables[12].update_table_status("paid")
        time.sleep(1)
        restaurant.clean_table("Fall1")

    print("\n--- Final assignment after more tables are free ---")
    restaurant.assign_table()
    restaurant.print_status()

Table name: August1, Capacity: 1 - 2
Table name: August2, Capacity: 1 - 2
Table name: August3, Capacity: 1 - 2
Table name: August4, Capacity: 1 - 2
Table name: August5, Capacity: 1 - 2
Table name: August6, Capacity: 1 - 2
Table name: Spring1, Capacity: 3 - 4
Table name: Spring2, Capacity: 3 - 4
Table name: Spring3, Capacity: 3 - 4
Table name: Spring4, Capacity: 3 - 4
Table name: Winter1, Capacity: 5 - 6
Table name: Winter2, Capacity: 5 - 6
Table name: Fall1, Capacity: 7 - 8
---------------------------


--- Adding initial customers to the queue ---
Enqueue: Customer: "Alice" / number of people: 2 / Type: reservation: Reservation Time: 2025-05-25 14:07:23.443469.
 No 1. Current queue: ['Customer: "Alice" / number of people: 2 / Type: reservation: Reservation Time: 2025-05-25 14:07:23.443469']
Enqueue: Customer: "Bob" / number of people: 4 / Type: reservation: Reservation Time: 2025-05-25 13:47:23.443499.
 No 2. Current queue: ['Customer: "Bob" / number of people: 4 / Type: reservation: 